In [25]:
from bfloat16 import bfloat16
import random

def bin_to_bfloat16(binary_str):
    sign = int(binary_str[0], 2)
    exp_with_bias = int(binary_str[1:9], 2)  # Convert exponent and apply bias
    mantissa = int(binary_str[9:], 2) / 128.0  # Convert mantissa and scale

    pos_0 = (sign == 0 and exp_with_bias == 0 and mantissa == 0)
    neg_0 = (sign == 1 and exp_with_bias == 0 and mantissa == 0)
    NaN = (exp_with_bias == 255 and mantissa != 0)

    float_value = float('nan') if NaN else 0.0 if pos_0 else -0.0 if neg_0 else ((-1)**sign) * (1 + mantissa) * (2 ** (exp_with_bias - 127))
    
    return bfloat16(float_value)

a_1234_div_10000 = bfloat16(1234.0/10000.0)
print(a_1234_div_10000.convert2bin())
print(a_1234_div_10000.convert2hex())
test_bf = bin_to_bfloat16('0011110111111100')
print(test_bf.convert2bin())
print(test_bf.convert2hex())

0011110111111100
3DFC
0011110111111100
3DFC


In [27]:
# def get_pos_inf():
#     return bin_to_bfloat16('0111111110000000')
# def get_neg_inf():
#     return bin_to_bfloat16('1111111110000000')
# def get_NaN():
#     return bin_to_bfloat16('0111111110000111')
# def get_pos_zero():
#     return bin_to_bfloat16('0000000000000000')
# def get_neg_zero():
#     return bin_to_bfloat16('1000000000000000')

def get_pos_inf():
    return bfloat16(float('inf'))
def get_neg_inf():
    return bfloat16(float('-inf'))
def get_NaN():
    return bfloat16(float('nan'))
def get_pos_zero():
    return bfloat16(0.0)
def get_neg_zero():
    return bfloat16(-0.0)

print(f'neg_inf: {get_neg_inf().convert2hex()}, {get_neg_inf().convert2bin()}')
print(f'pos_inf: {get_pos_inf().convert2hex()}, {get_pos_inf().convert2bin()}')
print(f'NaN:     {get_NaN().convert2hex()}, {get_NaN().convert2bin()}')
print(f'neg_0:   {get_neg_zero().convert2hex()}, {get_neg_zero().convert2bin()}')
print(f'pos_0:   {get_pos_zero().convert2hex()}, {get_pos_zero().convert2bin()}')


neg_inf: FF80, 1111111110000000
pos_inf: 7F80, 0111111110000000
NaN:     7FC0, 0111111111000000
neg_0:   8000, 1000000000000000
pos_0:   0000, 0000000000000000


In [36]:
def splice_bfloat_binary(str):
    sign = str[:1]
    exp = str[1:9]
    frac = str[9:]

    return sign, exp, frac

def mult_test(test_case='NaN'):
    if test_case == 'NaN':
        a = get_NaN()
        b = bfloat16(1234.0/10000.0)
        c = a * b
        s, e, f = splice_bfloat_binary(c.convert2bin())
        print(f'NaN x b   = {c.convert2hex()} = {s} | {e} | {f}')
    elif test_case == 'pos_inf_x_pos_0':
        a = get_pos_inf()
        b = get_pos_zero()
        c = a * b
        s, e, f = splice_bfloat_binary(c.convert2bin())
        print(f'+Inf x +0 = {c.convert2hex()} = {s} | {e} | {f}')
    elif test_case == 'pos_inf_x_neg_0':
        a = get_pos_inf()
        b = get_neg_zero()
        c = a * b
        s, e, f = splice_bfloat_binary(c.convert2bin())
        print(f'+Inf x -0 = {c.convert2hex()} = {s} | {e} | {f}')
    elif test_case == 'neg_inf_x_pos_0':
        a = get_neg_inf()
        b = get_pos_zero()
        c = a * b
        s, e, f = splice_bfloat_binary(c.convert2bin())
        print(f'-Inf x +0 = {c.convert2hex()} = {s} | {e} | {f}')
    elif test_case == 'neg_inf_x_neg_0':
        a = get_neg_inf()
        b = get_neg_zero()
        c = a * b
        s, e, f = splice_bfloat_binary(c.convert2bin())
        print(f'-Inf x -0 = {c.convert2hex()} = {s} | {e} | {f}')
    else:
        print('bad input')

mult_test(test_case='NaN')
mult_test(test_case='pos_inf_x_pos_0')
mult_test(test_case='pos_inf_x_neg_0')
mult_test(test_case='neg_inf_x_pos_0')
mult_test(test_case='neg_inf_x_neg_0')

NaN x b   = 7FC0 = 0 | 11111111 | 1000000
+Inf x +0 = FFC0 = 1 | 11111111 | 1000000
+Inf x -0 = FFC0 = 1 | 11111111 | 1000000
-Inf x +0 = FFC0 = 1 | 11111111 | 1000000
-Inf x -0 = FFC0 = 1 | 11111111 | 1000000
